# Leveraging GenAI Workflows

The below shows a very simple example of how to leverage LLMs in data analysis workflows. We will reuse a .csv file regarding flight data from earlier in the class.

First, we install the transformers package.

In [4]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 44.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1


To access and load Llama in a notebook environment, one must first authenticate their HuggingFace account.

In [5]:
from huggingface_hub import login
login(new_session=True)

The next cell loads a lightweight version of Llama onto the notebook. This is the model we will engage with for our simple data analysis task.

In [6]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
messages = [
    {"role": "user", "content": "Who are you?"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


I'm an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."<|eot_id|>


Now that the model is loaded, we prepare the file we are interested in. The file is a year of aggregated flight data from a previous lecture.

In [7]:
import pandas as pd
path = '/content/2015-summary.csv'  # uploading to notebook instance, path may vary
summary = pd.read_csv(path)
summary = summary.sort_values("count", ascending = False)
test = summary.head()

In [13]:
test

,DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
81,United States,United States,370002
214,United States,Canada,8483
115,Canada,United States,8399
139,United States,Mexico,7187
43,Mexico,United States,7140


Above, we load the aggregated data and sort it by the flight routes that are taken most frequently. Now, we share it with the loaded model and ask it to generate a brief report.

In [16]:
messages = [
    {"role": "user", "content":f"""
    The following includes some of the most common international flight routes within our airline. Please only list the international routes and briefly suggest what this means for our team's short term operations: {test}"""}
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens = 250)
print(f"\n{tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:])}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Based on the provided international flight routes, here are the destinations and their corresponding counts:

- United States: 
  - Canada: 8483
  - Mexico: 7187

These destinations indicate that your airline primarily operates flights to Canada and Mexico. Given the relatively high counts, it's likely that these routes are the most important and busiest for your airline.

For your team's short-term operations, considering these routes might require:

- Coordinating with airlines or ground handling companies to manage the increased demand for flights to Canada and Mexico.
- Ensuring sufficient aircraft and crew resources are allocated to these routes.
- Reviewing and adjusting your operational procedures to handle the increased volume of passengers and cargo.
- Possibly implementing measures to optimize flight schedules and routing to minimize delays and optimize fuel efficiency.

It's essential to assess your airline's specific operational needs and constraints before making any deci

The model does a decent job summarizing the data based on the prompt we give it. That said, in a notebook, something like this is not very scalable. In order to scale up, we'll need to house the model in something different.